<a href="https://colab.research.google.com/github/abhinavsb3/Pruning_for_Model_Distillation/blob/main/Layer_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python -m pip install --upgrade pip -q
!pip install transformers -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.9 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM

#Model = "HuggingFaceTB/SmolLM-360M-instruct"
#Model = "HuggingFaceTB/SmolLM-135M-instruct"

#Student Model
model_name = "HuggingFaceTB/SmolLM-135M"

model = AutoModelForCausalLM.from_pretrained(model_name)

print("original_model:")
print(model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

original_model:
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
    (rotary_emb

**LAYER PRUNING**

In [7]:
import torch
import math

def total_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

def create_pruned_lm(model_name, target_params):
  model = AutoModelForCausalLM.from_pretrained(model_name)

  print("Original_model:")
  print(model)

  original_params = total_parameters(model)
  print(f"Original model parameters: {original_params}")

  total_layers = len(model.model.layers)

  #Calculate number of layers to keep
  layers_to_keep = round((target_params / original_params) * total_layers)
  layers_to_prune = total_layers - layers_to_keep

  #Keep all layers except those right before the last layer
  final_layers = (
      list(model.model.layers[:total_layers - layers_to_prune - 1] +
           [model.model.layers[-1]])
  )

  model.model.layers = torch.nn.ModuleList(final_layers)

  model.config.num_hidden_layers = len(final_layers)

  print(".\nModifed model (PrunedLM)")
  print(model)

  new_params = total_parameters(model)
  print(f"The new pruned model parameters: {new_params:,}")

  reduction_percentage = (1 - new_params / original_params) * 100
  print(f"Size reduction: {reduction_percentage:.2f}%")

  return model


model_name = "HuggingFaceTB/SmolLM-135M"
target_params = 90_000_000

pruned_lm = create_pruned_lm(model_name, target_params)

modified_model_path = f"{model_name.split('/')[1]}-layer-pruned-{int(target_params/1000000)}M-raw"
pruned_lm.save_pretrained(modified_model_path)
print(f"\nTrelisLM-100M-Instruct saved to: {modified_model_path}")


Original_model:
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
    (rotary_emb

Push the Model to Hub


In [8]:
from huggingface_hub import login
login()
## my token

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def push_model_to_hub(local_model_path, original_model_name, repo_name):
  #Using the same tokenizer as original model
  tokenizer = AutoTokenizer.from_pretrained(original_model_name)

  #Load the modified model with ignore_mismatched_size
  model = AutoModelForCausalLM.from_pretrained(local_model_path,
                                        torch_dtype=torch.bfloat16,
                                        ignore_mismatched_sizes=True #This required for pushing the pruned model
                                        )

  #Push model to hub
  model.push_to_hub(repo_name)

  #Push the tokenizer to the hub
  tokenizer.push_to_hub(repo_name)

  print(f"Model and tokenizer pushed successfully to {repo_name}")

#Path to my local model is "modified_model_path"
local_model_path = modified_model_path

repository_name = f"abhinavv3/{local_model_path}"

original_model_name = "HuggingFaceTB/SmolLM-135M-Instruct"

#push the model
push_model_to_hub(local_model_path, original_model_name, repository_name)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...90M-raw/model.safetensors:  12%|#2        | 24.5MB /  198MB            

README.md: 0.00B [00:00, ?B/s]

Model and tokenizer pushed successfully to abhinavv3/SmolLM-135M-layer-pruned-90M-raw
